In [48]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict.load_from_disk("common_voice")

# Train the model using a small part of the training set
common_voice_train = common_voice["train"].select(range(1000))
common_voice_test = common_voice["test"].select(range(200))

In [49]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("dussen/whisper-small-hi")

config.json: 100%|██████████| 1.32k/1.32k [00:00<?, ?B/s]
model.safetensors: 100%|██████████| 967M/967M [05:31<00:00, 2.92MB/s] 
generation_config.json: 100%|██████████| 3.84k/3.84k [00:00<?, ?B/s]


In [50]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: Italian, split:  test",
    "language": "ita",
    "model_name": "Whisper Small - Italian",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [51]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="./whisper-small-ita",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=200,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps= 50,
    eval_steps= 50,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [52]:
from transformers import WhisperProcessor
from transformers import WhisperTokenizer

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")


In [53]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

metric = evaluate.load("wer")

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [54]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [55]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [56]:
trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 12%|█▎        | 25/200 [41:46<4:50:27, 99.59s/it] 

{'loss': 0.0128, 'learning_rate': 5e-06, 'epoch': 0.4}


 25%|██▌       | 50/200 [1:34:27<10:12:53, 245.16s/it]

{'loss': 0.0082, 'learning_rate': 1e-05, 'epoch': 0.79}


In [ ]:
# processor.save_pretrained("./dussen/whisper-small-ita")
trainer.push_to_hub(**kwargs)

events.out.tfevents.1701958893.Merlijn.12644.1: 100%|██████████| 8.04k/8.04k [00:00<00:00, 18.3kB/s]


'https://huggingface.co/dussen/whisper-small-hi/tree/main/'